In [1]:
# Imports
import os
import sys
import unidecode

In [2]:
# Setup tika
import tika
tika.initVM()
from tika import parser

In [5]:
os.getcwd()

'/home/alex/dev/michael/contraxsuite/lexpredict-contraxsuite-core/notebooks/embeddings/contracts'

In [14]:
# NLP/ML imports
sys.path.append("../../../lexnlp/")
import nlp.en.tokens
from nlp.en.segments.sentences import get_sentences

from gensim.models.word2vec import Word2Vec

/home/alex/dev/michael/contraxsuite/cve/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [9]:
# Setup default path for documents
document_path = "/home/alex/Desktop/test_folder/"
document_type = "*"

In [10]:
def build_file_list(path, extension=None):
    file_list = []
    for file_name in os.listdir(path):
        if os.path.isdir(os.path.join(path, file_name)):
            file_list.extend(build_file_list(os.path.join(path, file_name)))
        elif os.path.isfile(os.path.join(path, file_name)):
            if extension and file_name.lower().endswith(extension.lower()):
                file_list.append(os.path.join(path, file_name))
            else:
                file_list.append(os.path.join(path, file_name))
    return file_list

In [15]:
# Iterate through file list            
file_list = build_file_list(document_path)
file_list = file_list

sentence_list = []

print("training on {0} files...".format(len(file_list)))
for file_name in file_list:
    # Load document
    tika_response = tika.parser.from_file(os.path.join(document_path, file_name))
    try:
        tika_content = unidecode.unidecode(tika_response["content"])
    except KeyError as e:
        continue
    
    # Parse into sentences and tokens
    for sentence in get_sentences(tika_content):
        sentence_list.append(nlp.en.tokens.get_lemmas(sentence, lowercase=True))        

print("found {0} sentences.".format(len(sentence_list)))

training on 5 files...
found 13 sentences.


In [19]:
# Train w2v CBOW model
w2v_model_cbow = Word2Vec(sentence_list, vector_size=200, window=20, min_count=10, workers=2)
w2v_model_cbow.save("w2v_model_cbow")
print("cbow trained.")

# Train w2v CBOW model
w2v_model_sg = Word2Vec(sentence_list, vector_size=200, window=20, min_count=10, workers=2, sg=1)
w2v_model_sg.save("w2v_model_sg")
print("sg trained.")

RuntimeError: you must first build vocabulary before training the model

In [18]:
sentence_list = [list(i) for i in sentence_list]